
send voice on discord?


In [ ]:
import os
import io
import ffmpeg
import discord
import outetts
import asyncio

# Initialize Discord bot
TOKEN = os.getenv("BOT_TOKEN")
GUILD_ID = os.getenv("DEFAULT_GUILD_ID")
# VC_CHANNEL_ID = "YOUR_VOICE_CHANNEL_ID"
model_config = outetts.HFModelConfig_v2(
    model_path="OuteAI/OuteTTS-0.3-1B",
    tokenizer_path="OuteAI/OuteTTS-0.3-1B"
)

async def init_discord(token = TOKEN):

  # Initialize Outetts
  interface = outetts.InterfaceHF(model_version="0.3", cfg=model_config)
  speaker = interface.load_default_speaker(name="en_male_1")

  client = discord.Client(intents=discord.Intents.default())
  globals()['client'] = client
  #client.run(TOKEN)
  #client.event(connect_voice)
  await asyncio.gather(
    client.start(TOKEN),  # Start the bot
    connect_voice()  # Connect to VC
  )

async def connect_voice():
    client = globals()['client']
    print(f'Logged in as {client.user}')
    guild = discord.utils.get(client.guilds, id=int(GUILD_ID))
    vc_channel = discord.utils.get(guild.voice_channels) #, id=int(VC_CHANNEL_ID))
    if vc_channel:
        vc = await vc_channel.connect()
        await play_tts(vc)

async def play_tts(prompt, vc):
    # Generate speech
    gen_cfg = outetts.GenerationConfig(
        text=prompt,
        temperature=0.3,
        repetition_penalty=1.1,
        max_length=4096,
        speaker=speaker,
    )
    output = interface.generate(config=gen_cfg)

    # Convert Outetts output to a stream
    audio_buffer = io.BytesIO()
    output.save(audio_buffer)
    audio_buffer.seek(0)

    # Stream audio to Discord using FFmpeg
    ffmpeg_process = (
        ffmpeg.input('pipe:0', format='wav')
        .output('pipe:1', format='opus', acodec='libopus', ar='48000')
        .run_async(pipe_stdin=True, pipe_stdout=True, pipe_stderr=True)
    )

    ffmpeg_process.stdin.write(audio_buffer.read())
    ffmpeg_process.stdin.close()

    # Play the stream in Discord
    audio_source = discord.FFmpegPCMAudio(ffmpeg_process.stdout, pipe=True)
    vc.play(audio_source)

#__all__ = {
#  "init_discord": init_discord,
#  "connect_voice": connect_voice,
#  "play_tts": "play_tts",
#}

__all__ = init_discord
